## 2.VaR with Unknown Parameters.
### Edited by Xuyang Liu

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from  scipy.stats import norm , chi2

In [2]:
Data = pd.read_excel('AAPL_Data.xlsx',index_col=0)
Data.columns = ['AAPL']
logret = np.log(Data/Data.shift(1)).dropna()

In [3]:
alpha = 0.97
beta = 0.02
M = 125000

#### (a) Estimate VaR when sigma is unknown

In [4]:
# (1) For Empirical Distribution
capital = 1000000
Losslst = - capital * (np.exp(logret)-1)

In [5]:
mu = logret.mean()
sigma = (logret.var()*len(logret)/(len(logret)-1))**0.5
mu, sigma

(AAPL    0.001341
 dtype: float64,
 AAPL    0.012974
 dtype: float64)

In [6]:
VaR_EM = Losslst.quantile(alpha)
print('the VaR using Empirical distribution is about: ',VaR_EM[0])

the VaR using Empirical distribution is about:  24919.07770155716


In [7]:
# (2) For theoretical Formular
VaR_TH = capital * (1-np.exp(mu+sigma * norm.ppf(1-alpha)))
print('the VaR using theoretical formula is about: ',VaR_TH[0])

the VaR using theoretical formula is about:  22797.2651010363


In [8]:
# (3) CI for this VaR_TH
sigma_high = ((len(logret)-1)/chi2.ppf(0.5*beta, df=len(logret)-1))**0.5*sigma
sigma_low = ((len(logret)-1)/chi2.ppf(1-0.5*beta, df=len(logret)-1))**0.5*sigma
(sigma_low[0], sigma_high[0])

(0.01208463620803254, 0.013998385618034391)

In [9]:
VaR_high = capital * (1 - np.exp(mu - sigma_high * norm.ppf(alpha)))
VaR_low = capital * (1 - np.exp(mu - sigma_low * norm.ppf(alpha)))
(VaR_low[0],VaR_high[0])
print("The confidence interval for the theoretical VaR is (",VaR_low[0],",",VaR_high[0],")")

The confidence interval for the theoretical VaR is ( 21160.661298385676 , 24677.530906176526 )


#### (a) Estimate VaR when mu and sigma are unknown

In [10]:
sim_sigma = (len(logret-1) / np.random.chisquare(df=len(logret-1),size=M))**0.5 * sigma[0]
sim_mu = np.random.normal(loc=mu, scale=sim_sigma/np.sqrt(len(logret)))

In [11]:
sim_VaR_lst = capital*(1-np.exp(sim_mu+sim_sigma*norm.ppf(1-alpha)))
sim_VaR = np.mean(sim_VaR_lst)
print('the VaR using simulation method is about: ',sim_VaR)

the VaR using simulation method is about:  22833.11620657048


In [12]:
# For CI and A,B
A = np.quantile(sim_VaR_lst, 0.5*beta)
B = np.quantile(sim_VaR_lst, 1-0.5*beta)

In [13]:
print("The confidence interval (A, B) for the simulation VaR is (",A,",",B,")")

The confidence interval (A, B) for the simulation VaR is ( 20740.153675507692 , 25121.482477054185 )
